# Step-by-step run of alphazero self-play & training.


In [1]:
import os
import time
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players import alphazero
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game


from rgi.rgizero.common import TOKENS

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e"
os.makedirs(DATA_DIR, exist_ok=True)

%load_ext line_profiler

✅ Imports successful
Using device: mps


In [2]:
LOAD_MODEL = False
MODEL_SIZE = "small"  # "tiny" or "small" or"large" or "xl"
NUM_SIMULATIONS = 50

## Step 1: Set up history-wrapped game


In [3]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(base_game.all_actions()))
n_max_context = max_game_length + 2

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {base_game.__class__.__name__}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

✅ Using HistoryTrackingGame from module
Game: Connect4Game, Players: 2, Actions: [1, 2, 3, 4, 5, 6, 7]


## Step 2: Create or load model.


In [4]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer, ActionHistoryTransformerEvaluator
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}


def create_random_model(config: TransformerConfig, action_vocab_size, num_players,  seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed) # Ensure numpy operations are also seeded
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    model = ActionHistoryTransformer(config=config, action_vocab_size=action_vocab_size, num_players=num_players)
    model.to(device)
    return model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]

if LOAD_MODEL:
    raise NotImplementedError("Model loading not implemented")
else:
    model = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

# Step 3: Play games to generate training data


In [ ]:
reload_local_modules(verbose=False)
import asyncio
from tqdm.asyncio import tqdm

from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, QueuedNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async

async def play_games_async(num_games: int, player_factory):
    tasks = []
    async def create_player_and_create_game():
        t0 = time.time()
        player = player_factory()
        game_result = await play_game_async(game, [player, player])
        t1 = time.time()
        game_result['time'] = t1 - t0
        return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results


serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=1024, verbose=False)

master_rng = np.random.default_rng(42)
async_evaluator_factory = lambda: AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=1024, verbose=False)
async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=NUM_SIMULATIONS)

NUM_GAMES = 1000
print(f"Playing {NUM_GAMES} games, simulations={NUM_SIMULATIONS}, model_size={MODEL_SIZE}")
await async_evaluator.start()
results = asyncio.run(play_games_async(num_games=NUM_GAMES, player_factory=async_player_factory)) # 50 games, 4.3s
await async_evaluator.stop()

Playing 1000 games, simulations=50, model_size=small


100%|██████████| 1000/1000 [00:32<00:00, 30.78it/s]


In [ ]:
from collections import defaultdict, Counter

print("Winner by initial move:")
dd = defaultdict(Counter)
for result in results:
    dd[result['action_history'][0]][result['winner']] += 1
for action, counts in sorted(dd.items()):
    print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}")

## Resunts from 1000 connect4 games with random model.
# Winner by initial move:
#   a=1: n=121 win[1]=59.50% counts=Counter({1: 72, 2: 49})
#   a=2: n=139 win[1]=58.99% counts=Counter({1: 82, 2: 57})
#   a=3: n= 95 win[1]=63.16% counts=Counter({1: 60, 2: 35})
#   a=4: n=160 win[1]=78.75% counts=Counter({1: 126, 2: 34})
#   a=5: n=146 win[1]=65.07% counts=Counter({1: 95, 2: 51})
#   a=6: n=176 win[1]=61.36% counts=Counter({1: 108, 2: 68})
#   a=7: n=163 win[1]=53.37% counts=Counter({1: 87, 2: 76})

Winner by initial move:
  a=1: n=121 win[1]=59.50% counts=Counter({1: 72, 2: 49})
  a=2: n=139 win[1]=58.99% counts=Counter({1: 82, 2: 57})
  a=3: n= 95 win[1]=63.16% counts=Counter({1: 60, 2: 35})
  a=4: n=160 win[1]=78.75% counts=Counter({1: 126, 2: 34})
  a=5: n=146 win[1]=65.07% counts=Counter({1: 95, 2: 51})
  a=6: n=176 win[1]=61.36% counts=Counter({1: 108, 2: 68})
  a=7: n=163 win[1]=53.37% counts=Counter({1: 87, 2: 76})


## Step 3: Confirm we can read & write to trajectory_dataset


In [ ]:
all_actions = game.all_actions()
# num_players = game.num_players(state_0)
print(f"Vocab: {action_vocab}")

In [ ]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset, build_trajectory_loader
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

td_builder = TrajectoryDatasetBuilder(action_vocab)
for game_result in results:
    add_trajectory(game_result, action_vocab, td_builder)

td_builder.save(DATA_DIR, 'train_1000')

## Load dataset


In [ ]:
td = TrajectoryDataset(DATA_DIR, 'train_1000', 5)
td[0]


In [ ]:
loader = build_trajectory_loader(
    DATA_DIR, 'train_1000', block_size=5, batch_size=1,
    device_is_cuda=False, workers=4)

for batch in loader:
    print(batch)
    break

## Create model (random weights) and play a single game.


In [ ]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer, ActionHistoryTransformerEvaluator
from rgi.rgizero.models.transformer import TransformerConfig

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tiny_config: TransformerConfig = TransformerConfig(n_max_context=100, n_layer=2, n_head=2, n_embd=8)
tiny_model = ActionHistoryTransformer(config=tiny_config, action_vocab_size=vocab.vocab_size, num_players=num_players)
tiny_model.to(device)
tiny_evaluator = ActionHistoryTransformerEvaluator(tiny_model, device=device, block_size=5, vocab=vocab)


In [ ]:
legal_actions = game.legal_actions(state_0)
tiny_evaluator.evaluate(game, state_0, legal_actions)

In [ ]:
tiny_player = AlphazeroPlayer(game, tiny_evaluator)
tiny_game_result = play_game(game, [tiny_player, tiny_player])

tiny_game_result

In [ ]:
a_row = torch.rand(1, 2)
b_row = torch.rand(1, 2)
print(f'a_row: {a_row}')
print(f'b_row: {b_row}')
print(f'loss: {F.cross_entropy(a_row, b_row)}')

tile_shape = (3, 1)
a_tiled = torch.tile(a_row, tile_shape)
b_tiled = torch.tile(b_row, tile_shape)
print(f'a_tiled: {a_tiled}')
print(f'b_tiled: {b_tiled}')
print(f'loss: {F.cross_entropy(a_tiled, b_tiled)}')

print(f'loss: {F.cross_entropy(a_tiled, b_tiled, reduction="sum")}')
